In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from meteostat import Point, Daily
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('../pheno-data.csv')
df = df[(df['Phenophase_Description'] == 'Leaves (grasses)') & (df['Intensity_Value'] != '-9999')]
df['Abundance_Binary'] = df['Intensity_Value'].apply(lambda x: 1 if x == '75-94%' or x == '50-74%' or x == '95% or more' else 0)
df_filtered = df[['Observation_ID','Latitude', 'Longitude', 'Elevation_in_Meters', 'Phenophase_ID', 'Observation_Date', 'Abundance_Binary']]
df_filtered.head()

,Observation_ID,Latitude,Longitude,Elevation_in_Meters,Phenophase_ID,Observation_Date,Abundance_Binary
1754,1795824,32.250660,-110.946358,734,489,2013-01-28,1
1756,6838318,32.250660,-110.946358,734,489,2015-12-08,1
8332,2588139,32.212666,-111.001831,794,489,2013-07-23,1
8333,2588163,32.212666,-111.001831,794,489,2013-07-31,1
8334,2588187,32.212666,-111.001831,794,489,2013-08-06,1


In [3]:
scale = 4000 ## meters
label = df_filtered.Abundance_Binary.values ## labels
for days in range(24):
    
    pred = []
    
    for i in tqdm(range(len(df_filtered))):
            
        ## filter date
        start_date = df_filtered.Observation_Date.values[i]
        start_year = int(start_date[:4])
        start_month = int(start_date[5:7])
        start_day = int(start_date[8:10])
        start = datetime(start_year, start_month, start_day)
        
        end_date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=days)).strftime("%Y-%m-%d")
        end_year = int(end_date[:4])
        end_month = int(end_date[5:7])
        end_day = int(end_date[8:10])
        end = datetime(end_year, end_month, end_day)
        
        ## point location
        lat = df_filtered.Latitude.values[i]
        long = df_filtered.Longitude.values[i]
        elevation = df_filtered.Elevation_in_Meters.values[i]
        point = Point(lat, long, elevation)
        data = Daily(point, start, end)
        data = data.fetch()
        
        ## get precipitation time series data
        try:
            ts_prec = data['prcp'].values*0.0393701 ## 1mm = 0.0393701 inches        
            pred.append(1*(sum(ts_prec)>1.7))
        except:
            pred.append(0)
    
    tn, fp, fn, tp = confusion_matrix(pred, label).ravel()            
    acc = 100*(tn+tp)/len(label)
    fp_rate = 100*(fp)/len(label)
    fn_rate = 100*(fn)/len(label)
    
    print(f'Accumulative Days {days}.')
    print(f'Accuracy: {acc}%')
    print(f'FP: {fp_rate}%')
    print(f'FN: {fn_rate}%')

100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [00:42<00:00, 39.61it/s]


Accumulative Days 0.
Accuracy: 64.25206124852768%
FP: 35.74793875147232%
FN: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [00:44<00:00, 37.98it/s]


Accumulative Days 1.
Accuracy: 64.54652532391049%
FP: 35.39458186101296%
FN: 0.05889281507656066%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [00:47<00:00, 35.78it/s]


...

Accumulative Days 21.
Accuracy: 62.073027090694936%
FP: 29.38751472320377%
FN: 8.539458186101296%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [00:47<00:00, 35.60it/s]


Accumulative Days 22.
Accuracy: 61.48409893992933%
FP: 28.50412249705536%
FN: 10.011778563015312%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [00:48<00:00, 35.21it/s]

Accumulative Days 23.
Accuracy: 60.83627797408716%
FP: 28.386336866902237%
FN: 10.777385159010601%
